In [32]:
import ee

ee.Initialize()

In [79]:
# set up relevant parameters
import geopandas as gpd
import os
import matplotlib.pyplot as plt
wd = '/home/beeb/Insync/sei2112@columbia.edu/Google Drive/Columbia SusDev PhD/Research/My work/predictArsenic/'
os.chdir(wd)
outpath = 'data/raw/earthEngine/{}'
mouzapath = 'data/intermediate/placesToShapes/results.shp'
countrypath = 'data/raw/shapefiles/bgd_admbnda_adm{level}_bbs_20180410/bgd_admbnda_adm{level}_bbs_20180410.shp'

In [125]:
# load the shape file

mouzas = gpd.read_file(mouzapath).set_index('index')
# create the geometries in a format GEE can understand
mouzas['geoJsons'] = [i.__geo_interface__ for i in mouzas.geometry]

In [36]:
# set up satellite colleciton of interest
satellite = 'LANDSAT/LC08/C01/T1_SR'

# set up initial year
startYear = 2010;

# set up number of years we are going to be looking at
nYears = 5;

# set up months for the two rice growing seasons
aus = ['-06-15', '-07-15']
aman = ['-10-15',  '-11-15']

# set up band names appropriate for LANDSAT 8
nir = 'B4'
red = 'B3'

# calculate NDVI
def addNDVI(image):
    ndvi = image.normalizedDifference([nir, red]).rename('NDVI')
    return image.addBands(ndvi)

# function to filter collection by rice-growing season
def getSeason(whole, dates, year):
    startDate = str(year) + dates[0]
    endDate = str(year) + dates[1]
    return whole.filterDate(startDate, endDate)


# function to mask out cloudy pixels
def fixCloud(image):
    cloudScore = ee.Algorithms.Landsat.simpleCloudScore(image)
    mask = cloudScore.select(['cloud']).lte(20);
    masked = image.updateMask(mask);
    return masked


In [143]:
def export(img, poly, query):
    task_config = {
        'scale': 30,  
        'region': poly
    }

    task = ee.batch.Export.image.toDrive(image = img, scale = 30).start()


In [ ]:
task = ee.batch.Export.image.toDrive(
      image = img_cummulative.clip(region),
      folder = 'landsat',
      fileNamePrefix = band,
      scale = 30).start()  

In [129]:
def downloadSinglePoly(ausaman, poly, query):
    # initialise the whole collection
    wholeCollection = ee.ImageCollection(satellite).filterBounds(poly)
    # specify the season we're currently looking at
    collection = getSeason(wholeCollection, ausaman, startYear)
    # range through all years, add them to the current collection
    for year in range(startYear, startYear + nYears):
        collection = collection.merge(getSeason(wholeCollection, ausaman, year))
    # add NDVI, mask cloud and take median of all pixels
    collection = collection.map(addNDVI).map(fixCloud).median()
    export(collection, poly, query)

In [133]:

def downloadAllData(ausaman):
    
    # iterate through the various polygons and download the 
    for i in range(len(mouzas)): 
        if mouzas.geometry.iloc[i].type != 'MultiPolygon':
            print(mouzas.index[i])
            downloadSinglePoly(ausaman, mouzas['geoJsons'].iloc[i], mouzas.index[i])

In [144]:
downloadAllData(aus)

Laupala
Anardanga
Badan Haldarber
Baitpur
Chaitali
Chargram
Chhota Bishnupur
Dattakati
Dhanaganti
Joygachhi
Kapali Bandar
Kondala
Mauzardanga
Dema
Alakdia
Baniaganti
Betkhali
Bhatchhala
Debpara
Gabarkhali
Gaokhali
Gopalkati
Gotapara
Kaldia
Mukshait
Nataikhali
Patilakhali
Bara Raghunathpur
Chanpatala
Masidpur
Rahimabad Secondary School
Udkul
Deolbari
Gobardia
Guzihati
Karapara
Katua
Pater Para
Singrai
Bhatta Baliaghata
Chulkati
Dakshin Khanpur
Kismat Bhatta
Ranjitpur
Uttar Khanpur
Dari Rasulpur
Sugandhi Secondary Girls School
Sunagar
Bara Guni
Barabaria
Paschim Barabaria
Baniari
Char Dakatia
Kharamkhali
Khudari-Shyampara Rd
Uttar Khalisakhali
Aruabanni
Arulia
Chaudda Hazari
Kalsira
Karaterdia
Kharia
Kurmuni
Sabakhali
Suriganti
Char Saildaha
Hizla
Pirerabad
Char Saildaha
Ganga Channa
Machandapur (Ghararia)
Santoshpur
Umajuri
Hochla
Bahirdia Mansa
Joypur Betaga
Kismat Bighai (Niklepur)
Dhonputa-Maskata Road
Saittala
Surulia (Talbari)
Attaki
Brahman Rakdia
Hogaldanga
Fakirhat kathaltala
Pa